# Generate Vis for Fulton County Appeal Filings

In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import plotly.express as px
import pandas as pd

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

import map_utils

In [2]:
SHOW_MAP = False
SAVE_MAP = True
by_city = gpd.read_file("output/by_city.csv")
by_year = gpd.read_file("output/by_year.csv")
by_parcel = gpd.read_file("output/by_parcel.csv")
fulton_cities = gpd.read_file("data/fulton_cities.geojson")
geq_n_appeals = pd.read_csv("output/geq_n_appeals.csv")
# TODO chained indexing?
fulton_cities.loc[fulton_cities['Name'] == 'Atlanta', 'geometry'] = fulton_cities[fulton_cities['Name'].isin(['Atlanta', 'Fulton County'])].unary_union
fulton_cities = fulton_cities[fulton_cities['Name'] != 'Fulton County']

by_city = by_city.drop(by_city.index[-1])
by_year = by_year.drop(by_year.index[-1])
by_parcel = by_parcel.drop(by_parcel.index[-1])

In [3]:
def save_plot(fig, filename):
    fig.write_html(filename + '.html', full_html=False)
    # kaleido==0.1.0.post1
    # fig.write_image(filename + '.png')
    # saving png is not working

In [4]:
df = by_city
geo = fulton_cities
loc = 'City'
featureid = 'properties.Name'

## Generate map showing absolute number of appeals for each city

In [5]:
#| label: num_appeals
log = 10
color = "count_appeals_city"
hover_data = {"count_appeals_city": (":,", "Count")}
title = "Number of Appeals Filed in Each City of Fulton County (2011-2022)"

num_appeals = map_utils.gen_choropleth_log(
    log, df, geo, loc, featureid, color, hover_data, title, ","
)
if SHOW_MAP:
    num_appeals.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/num_appeals")

## Generate map showing percent of parcels filing an appeal in each city

In [6]:
#| label: pct_fulton_appeals
log = 10
color = "pct_appeals_in_city"
hover_data = {"pct_appeals_in_city": (":.2", "Percent")}
title = "Appeals Filed in Each City as % of Appeals Filed in Fulton County (2011-2022)"
postfix = "%"
# scale = 'ylorrd'
pct_fulton_appeals = map_utils.gen_choropleth_log(
    log, df, geo, loc, featureid, color, hover_data, title, postfix=postfix
)
if SHOW_MAP:
    pct_fulton_appeals.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/pct_appeals_of_fulton")

## Generate map with normalized metric (pct_appeals_in_city / pct_parcels_in_city); values closer to 1 are better

In [7]:
#| label: pct_appeals_over_per_parcels
color = "pct_appeals_over_pct_parcels"
hover_data = {"pct_appeals_over_pct_parcels": (":.2", "Ratio")}
title = "% Appeals / % Parcels Filed in Each City of Fulton County (2011-2022)"
colorbar_scale = dict(cmid=1)

normalized_city = map_utils.gen_choropleth(
    df,
    geo,
    loc,
    featureid,
    color,
    hover_data,
    title,
    str_format=",",
    scale="temps",
    colorbar_scale=colorbar_scale,
)
if SHOW_MAP:
    normalized_city.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/normalized_appeals")

## Generate map showing number of parcels with more than 1 appeal each city (and as percent)

In [8]:
color = 'percent_filing_geq_than_2_appeals'
hover_data = {'percent_filing_geq_than_2_appeals': (":.2", "Percent")}
title = "Percent of Parcels Filing More than 1 Appeal (2011-2022)"

geq_1_city = map_utils.gen_choropleth(df, geo, loc, featureid, color, hover_data, title, ',')

if SHOW_MAP:
    geq_1_city.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/geq_1_appeals")

In [9]:
color = 'percent_filing_geq_than_6_appeals'
hover_data = {'percent_filing_geq_than_6_appeals': (":.2", "Percent")}
title = "Percent of Parcels Filing More than 6 Appeals (2011-2022)"

geq_6_city = map_utils.gen_choropleth(df, geo, loc, featureid, color, hover_data, title, ',')

if SHOW_MAP:
    geq_6_city.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/geq_6_city")

In [10]:
color = 'percent_filing_geq_than_9_appeals'
hover_data = {'percent_filing_geq_than_9_appeals': (":.2", "Percent")}
title = "Percent of Parcels Filing More than 9 Appeals (2011-2022)"

geq_9_city = map_utils.gen_choropleth(df, geo, loc, featureid, color, hover_data, title, ',')

if SHOW_MAP:
    geq_9_city.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/geq_9_city")

## Generate scatter map showing number of parcels with more than 1 appeal

In [11]:
# Using map from FCS repo I made previously; will place in output folder

## Generate map showing number of repeat appeals each city (slider for 1 to 11 appeals)

In [12]:
#| label: animated_n_appeals
f = px.choropleth(
    geq_n_appeals,
    geojson=geo,
    locations="city",
    featureidkey=featureid,
    color="appeals",
    animation_frame="geq_n_appeals",
    animation_group="city",
    color_continuous_scale="ylorrd",
    hover_name="city",
    hover_data={"city": False, "geq_n_appeals": False, "appeals": ":.2f"},
    labels={"appeals": "Percent Filing Given Number of Appeals"},
)
f.update_geos(fitbounds="locations", visible=False)

f.update_layout(
    title=dict(
        text="Percent of Parcels Filing at Least 1 Appeal (2011-2022)",
        font=dict(size=20),
        automargin=True,
        yref="paper",
    ),
    coloraxis_colorbar=dict(
        title="",
    ),
    title_x=0.5,
    margin={"r": 0, "t": 35, "l": 0, "b": 0},
    width=1200,
    height=630,
    sliders=[{"currentvalue": {"prefix": "Number of Appeals: "}}],
)

for k in range(0, len(f.frames)):
    f.frames[k]["layout"].update(
        title_text=f"Percent of Parcels Filing at Least {k+1} Appeal(s) (2011-2022)"
    )
if SHOW_MAP:
    f.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/animated_n_appeals")

## Generate map showing number of repeat appeals each parcel (like old map) or with slider

In [13]:
NotImplemented

NotImplemented

## Generate map showing number of successful appeals each city

In [14]:
#| label: success_city
log = 20
color = 'success_count_city'
hover_data = {'success_count_city': (":,", "Successful Appeals")}
title = "Number of Successful Appeals in Each City of Fulton County (2011-2022)"

success_city = map_utils.gen_choropleth_log(log, df, geo, loc, featureid, color, hover_data, title, ',')
if SHOW_MAP:
    success_city.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/num_successful_appeals")

## Generate map showing percent of successful appeals each city

In [15]:
#| label: percent_success
log = 2
color = 'success_rate_city'
hover_data = {'success_rate_city': (":.1f", "Successful Appeals")}
title = "Success Rate of Appeals in Each City of Fulton County (2011-2022)"

pct_success_city = map_utils.gen_choropleth_log(log, df, geo, loc, featureid, color, hover_data, title, ',')
if SHOW_MAP:
    pct_success_city.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/pct_successful_appeals")

## relationship between number of appeals and success (points where size represents number of appeals, color represents percent success)

In [16]:
# TODO need to make a DF where geometry is the centroid of each county, plot that as overlay
NotImplemented

NotImplemented

## Total amount appealed each city, revenue lost each city

In [17]:
df['sum_appealed_city'] = df['sum_appealed_city'].astype(float) / 1000000000

In [18]:
#| label: sum_appealed
log = 10
color = "sum_appealed_city"
hover_data = {
    "sum_appealed_city": (":,.2f", "Total sum appealed"),
    "lost_revenue_city": (":,.2f", "Reduction in tax revenue"),
}
title = "Total Sum Appealed in Each City of Fulton County (2011-2022, in Billion $)"
postfix = "B"

sum_appealed = map_utils.gen_choropleth_log(
    log, df, geo, loc, featureid, color, hover_data, title, postfix=postfix
)
if SHOW_MAP:
    sum_appealed.show()
if SAVE_MAP:
    save_plot(num_appeals, "output/sum_appealed")